In [1]:
raw_notes = [
    "Pt presents with dental caries and tooth pain.",
    "Patient reports no tooth pain.",
    "Caries noted on molar 17.",
    "Follow-up next week."
]

In [1]:
import spacy
from negspacy.negation import Negex

nlp = spacy.load("en_core_web_sm")

# 加入否定检测
nlp.add_pipe(        #把一个组件，按顺序插入到 NLP 流水线中
    "negex",           #negex = Negex(nlp)
    config={
        "ent_types": ["SYMPTOM", "DISEASE"]
    },
    last=True   #把 negex 放在 pipeline 的最后
)

In [2]:
SYMPTOMS = ["pain", "tooth pain", "sensitivity"]
DISEASES = ["caries", "dental caries", "decay"]
BODY_PARTS = ["tooth", "molar", "molar 17"]

def extract_medical_info(text):
    doc = nlp(text)

    result = {
        "text": text,
        "symptoms": [],
        "diseases": [],
        "body_parts": []
    }

    lowered = text.lower()

    # 1️⃣ 症状 + 否定
    for s in SYMPTOMS:
        if s in lowered:
            negated = any(
                token.text.lower() in ["no", "denies"]
                for token in doc
                if token.i < len(doc) - 1
            )
            result["symptoms"].append({
                "name": s,
                "negated": negated
            })

    # 2️⃣ 疾病
    for d in DISEASES:
        if d in lowered:
            result["diseases"].append(d)

    # 3️⃣ 部位
    for b in BODY_PARTS:
        if b in lowered:
            result["body_parts"].append(b)

    return result

In [4]:
raw_notes = [
    "Pt presents with dental caries and tooth pain.",
    "Patient reports no tooth pain.",
    "Caries noted on molar 17.",
    "Follow-up next week."
]

import pandas as pd

records = []

for note in raw_notes:
    info = extract_medical_info(note)
    records.append(info)

df = pd.DataFrame(records)
print(df)

                                             text  \
0  Pt presents with dental caries and tooth pain.   
1                  Patient reports no tooth pain.   
2                       Caries noted on molar 17.   
3                            Follow-up next week.   

                                            symptoms                 diseases  \
0  [{'name': 'pain', 'negated': False}, {'name': ...  [caries, dental caries]   
1  [{'name': 'pain', 'negated': True}, {'name': '...                       []   
2                                                 []                 [caries]   
3                                                 []                       []   

          body_parts  
0            [tooth]  
1            [tooth]  
2  [molar, molar 17]  
3                 []  


In [5]:
import spacy

# 🚀 1. 加载医疗 NER 模型（scispaCy 示例）
# 注意：en_core_sci_md / en_ner_bc5cdr_md 可以用 scispaCy 安装
# pip install scispacy
# pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/en_ner_bc5cdr_md-0.5.1.tar.gz
nlp = spacy.load("en_ner_bc5cdr_md")

# 2️⃣ 医院自定义词表（CSV 也可以加载，这里简化为列表）
SYMPTOMS = ["pain", "tooth pain", "sensitivity"]
DISEASES = ["caries", "dental caries", "decay"]
BODY_PARTS = ["tooth", "molar", "molar 17"]

# 3️⃣ 否定关键词列表
NEGATIONS = ["no", "denies", "without", "not"]

# ----------------------------
# 核心函数
# ----------------------------
def extract_medical_info(text):
    doc = nlp(text)
    lowered = text.lower()

    result = {
        "text": text,
        "symptoms": [],
        "diseases": [],
        "body_parts": []
    }

    # 🔹 1. 使用模型 NER 识别实体
    for ent in doc.ents:
        label = ent.label_.lower()
        name = ent.text.lower()

        # 检查是否被否定
        negated = any(tok.text.lower() in NEGATIONS for tok in doc if tok.i < ent.start)

        if label == "disease":
            result["diseases"].append(name)
        elif label == "symptom":
            result["symptoms"].append({"name": name, "negated": negated})
        # 部位模型可能没有，后续用词表补充

    # 🔹 2. 词表兜底匹配（模型可能识别不到）
    # 症状
    for s in SYMPTOMS:
        if s in lowered and s not in [e["name"] for e in result["symptoms"]]:
            negated = any(neg in lowered for neg in NEGATIONS)
            result["symptoms"].append({"name": s, "negated": negated})

    # 疾病
    for d in DISEASES:
        if d in lowered and d not in result["diseases"]:
            result["diseases"].append(d)

    # 部位
    for b in BODY_PARTS:
        if b in lowered and b not in result["body_parts"]:
            result["body_parts"].append(b)

    return result

# ----------------------------
# 测试文本
# ----------------------------
texts = [
    "Patient reports tooth pain but denies swelling.",
    "No evidence of dental caries, patient complains of sensitivity.",
    "Molar 17 shows signs of decay."
]

# 输出结构化信息
for t in texts:
    info = extract_medical_info(t)
    print(info)

OSError: [E050] Can't find model 'en_ner_bc5cdr_md'. It doesn't seem to be a Python package or a valid path to a data directory.